# Connection to DB

In [5]:
from sqlalchemy import create_engine
from sshtunnel import SSHTunnelForwarder

In [6]:
creds = {
    'DB_HOST': 'liradb.compute.dtu.dk',
    'DB_PORT': 5435,
    'DB_USER': 'guest',
    'DB_PASS': 'V2GjxQVn',
    'DB_NAME': 'postgres',
    'SSH_HOST': 'thinlinc.compute.dtu.dk',
    'SSH_USER': 's174420',  # SET YOUR USER HERE
    'SSH_PASS': open('creds.json').readline()
}

In [7]:
print('Connecting to the SSH Tunnel...')

ssh_tunnel = SSHTunnelForwarder(
    ssh_address_or_host=(creds['SSH_HOST'], 22),
    ssh_username=creds["SSH_USER"],
    ssh_password=creds['SSH_PASS'],
    remote_bind_address=(creds["DB_HOST"], creds['DB_PORT'])
)

ssh_tunnel.start()
print('Success')

Connecting to the SSH Tunnel...


In [12]:
print('Connecting to the PostgreSQL Database...')

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db}'.format(
    host='localhost',
    port=ssh_tunnel.local_bind_port,
    user=creds["DB_USER"],
    password=creds["DB_PASS"],
    db=creds["DB_NAME"]
))

conn = engine.connect()
print('Success')

Connecting to the PostgreSQL Database...


# Data processing

## Fetching trips

In [17]:
import pandas as pd

In [18]:
df_trips = pd.read_sql('select * from "Trips"', conn)

In [89]:
df_filter = df_trips.query('StartPositionLat not in ["0", 0, None]')

In [90]:
df_filter

,TripId,TaskId,StartTimeUtc,EndTimeUtc,StartPositionLat,StartPositionLng,StartPositionDisplay,EndPositionLat,EndPositionLng,EndPositionDisplay,Duration,DistanceKm,FK_Device,Created_Date,Updated_Date,Fully_Imported
0,a1367e81-c447-450c-9e6f-5e65fc519baf,1481,2020-04-16 04:36:34.503126+00:00,2020-04-16 04:41:03.419459+00:00,55.747610,12.498290,"{""city"":""Søborg"",""country"":""Denmark"",""county"":...",55.759070,12.519820,"{""city"":""Jægersborg"",""country"":""Denmark"",""coun...",2020-04-30 00:04:28.916333+00:00,0.495350,8051a60b-84f8-4c7e-9e6a-cd57bcf52ebe,2020-04-30 09:14:06.529764+00:00,0001-01-01 00:00:00+00:00,True
1,d84a0a79-de2b-4c7d-a2f0-c0c197834d6a,1482,2020-04-16 04:45:18.428042+00:00,2020-04-16 04:53:11.524610+00:00,55.699640,12.543980,"{""city"":""Copenhagen"",""country"":""Denmark"",""coun...",55.739610,12.548740,"{""city"":""Jægersborg"",""country"":""Denmark"",""coun...",2020-04-30 00:07:53.096568+00:00,5.420969,9491599b-0e08-438c-a7cb-39b10f34b2de,2020-04-30 15:38:56.489564+00:00,0001-01-01 00:00:00+00:00,True
2,e4968285-7440-4e73-9091-bc939b078e2f,1483,2020-04-16 05:46:11.871208+00:00,2020-04-16 05:53:46.213747+00:00,55.752600,12.560390,"{""city"":""Klampenborg"",""house_number"":""14"",""cou...",55.749840,12.500590,"{""city"":null,""country"":null,""county"":null,""sub...",2020-04-30 00:07:34.342539+00:00,4.062899,6513ab9d-66f0-4701-a2fb-a54dbaf12151,2020-04-30 16:16:37.923008+00:00,0001-01-01 00:00:00+00:00,True
3,7a25c6cc-cb2f-48f2-8569-681b93fc8d6b,1485,2020-04-16 06:29:12.638582+00:00,2020-04-16 06:42:25.249888+00:00,55.739600,12.548750,"{""city"":""Jægersborg"",""country"":""Denmark"",""coun...",55.679270,12.573230,"{""city"":""Copenhagen"",""country"":""Denmark"",""coun...",2020-04-30 00:13:12.611306+00:00,4.914425,9491599b-0e08-438c-a7cb-39b10f34b2de,2020-04-30 21:06:19.783617+00:00,0001-01-01 00:00:00+00:00,True
4,57ef0674-38fa-4919-a3f8-9dd687958378,1486,2020-04-16 10:27:06.503493+00:00,2020-04-16 10:43:28.233754+00:00,55.702080,12.454350,"{""city"":""Islev"",""house_number"":""418"",""country""...",55.626060,12.482680,"{""city"":""Avedøre Landsby"",""country"":""Denmark"",...",2020-05-04 00:16:21.730261+00:00,8.910680,bded9c90-025e-460a-9c30-b65a29d07b91,2020-05-04 11:21:24.298806+00:00,0001-01-01 00:00:00+00:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,06c4025f-c819-4960-9f74-da79420ceb98,4971,2020-09-09 15:45:43.103070+00:00,2020-09-09 18:37:22.213877+00:00,55.707640,12.568470,"{""city"":null,""country"":null,""county"":null,""sub...",None,None,"{""city"":null,""country"":null,""county"":null,""sub...",2020-10-06 02:51:39.110807+00:00,0.000000,85d44a66-4ecb-4fdc-91be-0a18b2d7ebdb,2020-10-06 00:37:58.546293+00:00,0001-01-01 00:00:00+00:00,False
641,eb9f06ce-26a6-4fc7-b4b9-14b52489b0d1,4000,2020-07-29 16:12:41.100782+00:00,2020-07-29 16:41:28.007266+00:00,55.669600,12.516670,"{""city"":null,""house_number"":""2"",""country"":""Den...",55.762640,12.437520,"{""city"":""Bagsværd"",""country"":""Denmark"",""county...",2020-10-12 00:28:46.906484+00:00,33.659322,85d44a66-4ecb-4fdc-91be-0a18b2d7ebdb,2020-10-12 09:20:54.061173+00:00,0001-01-01 00:00:00+00:00,True
642,46a1f2c3-d80f-4265-bf23-7b44d12c811d,3989,2020-07-31 14:32:24.340410+00:00,2020-07-31 14:39:12.637268+00:00,55.683830,12.588520,"{""ntk_geocode_time"":49,""city"":""Copenhagen"",""st...",55.678750,12.587190,"{""ntk_geocode_time"":120,""city"":""Copenhagen"",""s...",2020-10-12 00:06:48.296858+00:00,1.307534,eadc9490-e9de-4219-ba29-03472c74cefd,2020-10-12 12:09:30.882448+00:00,0001-01-01 00:00:00+00:00,True
643,2d593e5c-a958-4274-afa5-dbd2844086e8,4884,2020-09-05 13:56:17.481761+00:00,2020-09-05 15:05:05.951816+00:00,55.673780,12.570090,{},55.644360,12.551890,{},2020-10-12 01:08:48.470055+00:00,600.725614,187c70cc-4eff-4307-8157-ac0fa0102f43,2020-10-12 12:11:53.589292+00:00,0001-01-01 00:00:00+00:00,False


In [ ]:
import json

In [91]:
failed = []

for trip, row in enumerate(df_filter.StartPositionDisplay):
    try:
        json.loads(row)
    except Exception as e:
        print(e, trip)
        failed.append(trip)

In [92]:
df_filter.iloc[failed, :].query('StartPositionLat == "0"')

,TripId,TaskId,StartTimeUtc,EndTimeUtc,StartPositionLat,StartPositionLng,StartPositionDisplay,EndPositionLat,EndPositionLng,EndPositionDisplay,Duration,DistanceKm,FK_Device,Created_Date,Updated_Date,Fully_Imported


In [97]:
import json

extracted_columns = pd.json_normalize(df_filter.StartPositionDisplay.apply(json.loads))

In [100]:
df_filter.join(extracted_columns)

,TripId,TaskId,StartTimeUtc,EndTimeUtc,StartPositionLat,StartPositionLng,StartPositionDisplay,EndPositionLat,EndPositionLng,EndPositionDisplay,...,street_type,street_name,provider,precision,longitude,street,postal_code,address,latitude,type
0,a1367e81-c447-450c-9e6f-5e65fc519baf,1481,2020-04-16 04:36:34.503126+00:00,2020-04-16 04:41:03.419459+00:00,55.747610,12.498290,"{""city"":""Søborg"",""country"":""Denmark"",""county"":...",55.759070,12.519820,"{""city"":""Jægersborg"",""country"":""Denmark"",""coun...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d84a0a79-de2b-4c7d-a2f0-c0c197834d6a,1482,2020-04-16 04:45:18.428042+00:00,2020-04-16 04:53:11.524610+00:00,55.699640,12.543980,"{""city"":""Copenhagen"",""country"":""Denmark"",""coun...",55.739610,12.548740,"{""city"":""Jægersborg"",""country"":""Denmark"",""coun...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e4968285-7440-4e73-9091-bc939b078e2f,1483,2020-04-16 05:46:11.871208+00:00,2020-04-16 05:53:46.213747+00:00,55.752600,12.560390,"{""city"":""Klampenborg"",""house_number"":""14"",""cou...",55.749840,12.500590,"{""city"":null,""country"":null,""county"":null,""sub...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7a25c6cc-cb2f-48f2-8569-681b93fc8d6b,1485,2020-04-16 06:29:12.638582+00:00,2020-04-16 06:42:25.249888+00:00,55.739600,12.548750,"{""city"":""Jægersborg"",""country"":""Denmark"",""coun...",55.679270,12.573230,"{""city"":""Copenhagen"",""country"":""Denmark"",""coun...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57ef0674-38fa-4919-a3f8-9dd687958378,1486,2020-04-16 10:27:06.503493+00:00,2020-04-16 10:43:28.233754+00:00,55.702080,12.454350,"{""city"":""Islev"",""house_number"":""418"",""country""...",55.626060,12.482680,"{""city"":""Avedøre Landsby"",""country"":""Denmark"",...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,06c4025f-c819-4960-9f74-da79420ceb98,4971,2020-09-09 15:45:43.103070+00:00,2020-09-09 18:37:22.213877+00:00,55.707640,12.568470,"{""city"":null,""country"":null,""county"":null,""sub...",None,None,"{""city"":null,""country"":null,""county"":null,""sub...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,eb9f06ce-26a6-4fc7-b4b9-14b52489b0d1,4000,2020-07-29 16:12:41.100782+00:00,2020-07-29 16:41:28.007266+00:00,55.669600,12.516670,"{""city"":null,""house_number"":""2"",""country"":""Den...",55.762640,12.437520,"{""city"":""Bagsværd"",""country"":""Denmark"",""county...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,46a1f2c3-d80f-4265-bf23-7b44d12c811d,3989,2020-07-31 14:32:24.340410+00:00,2020-07-31 14:39:12.637268+00:00,55.683830,12.588520,"{""ntk_geocode_time"":49,""city"":""Copenhagen"",""st...",55.678750,12.587190,"{""ntk_geocode_time"":120,""city"":""Copenhagen"",""s...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,2d593e5c-a958-4274-afa5-dbd2844086e8,4884,2020-09-05 13:56:17.481761+00:00,2020-09-05 15:05:05.951816+00:00,55.673780,12.570090,{},55.644360,12.551890,{},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Fetching measurements

In [107]:
MEASUREMENT_TYPES = [
    'obd.rpm',  # Engine RPM
    'obd.trq_eff',  # Mean effective torque
    'obd.whl_trq_pot_ri',  # Total potential resistive
    'obd.rpm_elec',  # Electric Engine RPM
    'obd.trac_cons',  # Traction instant consumption
    'obd.cons_avg',  # Average consumption
    'obd.trip_cons_avg',  # Average trip consumption
    'obd.ww_f_req',  # Front wiping request
    'obd.temp',  # Temperature internal
    'obd.temp_ext',  # Temperature external
    'obd.rpm_fl',  # RPM Front Left
    'obd.rpm_fr',  # RPM Front Right
    'obd.rpm_rl',  # RPM Rear Left
    'obd.rpm_rr',  # RPM Rear Right
    'obd.spd_veh',  # Vehicle speed
]

In [ ]:
measurement_dict = {}


In [118]:
for measurement_type in MEASUREMENT_TYPES:
    if measurement_type not in measurement_dict.keys():
        print(f'Fetching data for {measurement_type}')
        sql = f'select * from "Measurements" where "T" = \'{measurement_type}\''
        df_temp = pd.read_sql(sql, conn)
        extracted_columns = pd.json_normalize(df_temp.message.apply(json.loads))
        measurement_dict[measurement_type] = df_temp.join(extracted_columns)

Fetching data for obd.rpm
Extracting columns
Joining
Done
Fetching data for obd.trq_eff
Extracting columns
Joining
Done
Fetching data for obd.whl_trq_pot_ri
Extracting columns
Joining
Done
Fetching data for obd.rpm_elec
Extracting columns
Joining
Done
Fetching data for obd.trac_cons
Extracting columns
Joining
Done
Fetching data for obd.cons_avg
Extracting columns
Joining
Done
Fetching data for obd.trip_cons_avg
Extracting columns
Joining
Done
Fetching data for obd.ww_f_req
Extracting columns
Joining
Done
Fetching data for obd.temp
Extracting columns
Joining
Done
Fetching data for obd.temp_ext
Extracting columns
Joining
Done
Fetching data for obd.rpm_fl
Extracting columns
Joining
Done
Fetching data for obd.rpm_fr
Extracting columns
Joining
Done
Fetching data for obd.rpm_rl
Extracting columns
Joining
Done
Fetching data for obd.rpm_rr
Extracting columns
Joining
Done
Fetching data for obd.spd_veh
Extracting columns
Joining
Done


In [120]:
measurement_dict['obd.rpm']

,MeasurementId,TS_or_Distance,T,lat,lon,message,isComputed,FK_Trip,FK_MeasurementType,Created_Date,Updated_Date,id,@vid,start_time_utc,end_time_utc,@t,@ts,@uid,@rec,obd.rpm.value
0,eff5cb28-90ab-4954-b994-cdece1b316da,2020-04-30 05:38:54.196125+00:00,obd.rpm,55.623543,12.575116,"{""id"":""00000000-0000-0000-0000-000000000000"",""...",True,19eed7c3-5632-4e3b-939c-cbe6bcdbac44,6e151a29-8e4c-4ed1-bbb1-45c9473ae86f,2020-05-15 13:17:06.706730+00:00,0001-01-01 00:00:00+00:00,00000000-0000-0000-0000-000000000000,6,0001-01-01T00:00:00+00:00,0001-01-01T00:00:00+00:00,obd.rpm,2020-04-30T05:38:54.196125+00:00,1dc469ed-988b-da41-7f9b-a466e1ddc211,2020-04-30T05:41:10.572759+00:00,0.0
1,40ec4d9c-56be-4dae-9ece-8ecd590270e3,2020-04-30 05:39:17.021413+00:00,obd.rpm,55.688725,12.575454,"{""id"":""00000000-0000-0000-0000-000000000000"",""...",True,87674044-59f3-4d78-b046-907b6a2a02ed,6e151a29-8e4c-4ed1-bbb1-45c9473ae86f,2020-05-15 13:18:04.163193+00:00,0001-01-01 00:00:00+00:00,00000000-0000-0000-0000-000000000000,8,0001-01-01T00:00:00+00:00,0001-01-01T00:00:00+00:00,obd.rpm,2020-04-30T05:39:17.021413+00:00,5fcc58d9-f372-63db-56f4-3c4586d2a0ec,2020-04-30T05:44:23.772875+00:00,2540.0
2,072f318d-2748-4a22-bf81-64a289387a8d,2020-04-30 05:39:19.009654+00:00,obd.rpm,55.688570,12.575530,"{""id"":""00000000-0000-0000-0000-000000000000"",""...",True,87674044-59f3-4d78-b046-907b6a2a02ed,6e151a29-8e4c-4ed1-bbb1-45c9473ae86f,2020-05-15 13:18:04.844187+00:00,0001-01-01 00:00:00+00:00,00000000-0000-0000-0000-000000000000,8,0001-01-01T00:00:00+00:00,0001-01-01T00:00:00+00:00,obd.rpm,2020-04-30T05:39:19.009654+00:00,5fcc58d9-f372-63db-56f4-3c4586d2a0ec,2020-04-30T05:44:23.77317+00:00,2550.0
3,97668f71-002d-4af9-b960-fabf49875cde,2020-04-30 05:39:35.942546+00:00,obd.rpm,55.688843,12.575107,"{""id"":""00000000-0000-0000-0000-000000000000"",""...",True,87674044-59f3-4d78-b046-907b6a2a02ed,6e151a29-8e4c-4ed1-bbb1-45c9473ae86f,2020-05-15 13:18:08.969742+00:00,0001-01-01 00:00:00+00:00,00000000-0000-0000-0000-000000000000,8,0001-01-01T00:00:00+00:00,0001-01-01T00:00:00+00:00,obd.rpm,2020-04-30T05:39:35.942546+00:00,5fcc58d9-f372-63db-56f4-3c4586d2a0ec,2020-04-30T05:44:23.774291+00:00,2560.0
4,eec6b4f9-6f91-4fe6-bfc3-ab68a38a12fb,2020-04-30 05:39:31.993533+00:00,obd.rpm,55.688522,12.575373,"{""id"":""00000000-0000-0000-0000-000000000000"",""...",True,87674044-59f3-4d78-b046-907b6a2a02ed,6e151a29-8e4c-4ed1-bbb1-45c9473ae86f,2020-05-15 13:18:07.718682+00:00,0001-01-01 00:00:00+00:00,00000000-0000-0000-0000-000000000000,8,0001-01-01T00:00:00+00:00,0001-01-01T00:00:00+00:00,obd.rpm,2020-04-30T05:39:31.993533+00:00,5fcc58d9-f372-63db-56f4-3c4586d2a0ec,2020-04-30T05:44:23.773931+00:00,2570.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145018,212d372c-86d5-4667-8762-71311b02ba63,2020-04-30 05:27:43.470305+00:00,obd.rpm,55.618330,12.580750,"{""id"":""00000000-0000-0000-0000-000000000000"",""...",True,19eed7c3-5632-4e3b-939c-cbe6bcdbac44,6e151a29-8e4c-4ed1-bbb1-45c9473ae86f,2020-05-15 13:11:46.605536+00:00,0001-01-01 00:00:00+00:00,00000000-0000-0000-0000-000000000000,6,0001-01-01T00:00:00+00:00,0001-01-01T00:00:00+00:00,obd.rpm,2020-04-30T05:27:43.470305+00:00,1dc469ed-988b-da41-7f9b-a466e1ddc211,2020-04-30T05:29:03.938973+00:00,3230.0
145019,814ff7fc-744b-4c24-afc2-ebb485cb5350,2020-04-30 05:27:15.053642+00:00,obd.rpm,55.618492,12.583550,"{""id"":""00000000-0000-0000-0000-000000000000"",""...",True,19eed7c3-5632-4e3b-939c-cbe6bcdbac44,6e151a29-8e4c-4ed1-bbb1-45c9473ae86f,2020-05-15 13:11:40.168817+00:00,0001-01-01 00:00:00+00:00,00000000-0000-0000-0000-000000000000,6,0001-01-01T00:00:00+00:00,0001-01-01T00:00:00+00:00,obd.rpm,2020-04-30T05:27:15.053642+00:00,1dc469ed-988b-da41-7f9b-a466e1ddc211,2020-04-30T05:29:03.936862+00:00,3170.0
145020,14d1944c-e48c-4fef-889f-ad5257868790,2020-04-30 05:27:11.435722+00:00,obd.rpm,55.618668,12.583874,"{""id"":""00000000-0000-0000-0000-000000000000"",""...",True,19eed7c3-5632-4e3b-939c-cbe6bcdbac44

# Visualization

In [322]:
conn = engine.connect()

In [323]:
import folium
import random

In [324]:
color_options = ['beige', 'black', 'blue', 'cadetblue', 'darkblue', 'darkgreen', 'darkpurple', 'darkred', 'gray', 'green', 'lightblue', 'lightgray',
                 'lightgreen', 'lightred', 'orange', 'pink', 'purple', 'red', 'white']

In [325]:
def get_trips(trips):
    sql = f'select * from "Measurements" where "FK_Trip" in {trips} and "T" = \'obd.acc_yaw\''
    df_plot = pd.read_sql(sql, conn).sort_values(['FK_Trip', 'TS_or_Distance'], ascending=True)
    return df_plot

In [326]:
num_trips = 50

df_plot = get_trips(tuple(df_filter.TripId[:num_trips].tolist()))

In [327]:
cph_lat = 55.676098
cph_lon = 12.568337

# Lat, Lon input
map_cph = folium.Map([cph_lat, cph_lon],
                     zoom_start=13,
                     tiles='Stamen Terrain')

num_trips = 50

for i, trip in enumerate(df_plot.FK_Trip.unique()):
    color = random.choice(color_options)

    print(f'Adding trip: {trip} with color {color}')
    try:
        data = [[row.lat, row.lon] for idx, row in df_plot[(df_plot.FK_Trip == trip)].iterrows()]
        folium.PolyLine(data, color=color, weight=5, popup=f"Trip ID: {trip}").add_to(map_cph)
    except:
        pass

map_cph

# HeatMap(heat_data,
#         radius=10).add_to(map_cph)


Adding trip: 0b79ca1c-354a-4979-8233-fba841cbf958 with color cadetblue
Adding trip: 138a07c9-fefb-4cbb-a2dd-a96d4b0b4f67 with color white
Adding trip: 17a2bc2c-7a25-4d73-b710-2b1aae2a97db with color pink
Adding trip: 1e360203-d84c-443e-9a94-8c9c78fe2e02 with color green
Adding trip: 1f508125-a6e5-411b-8335-e1436ddeb94d with color darkgreen
Adding trip: 20dbd3f4-2acb-4279-9b6d-e02d28e9f42b with color pink
Adding trip: 308ff659-545a-4dfd-a28f-cd4e91d4d7a0 with color red
Adding trip: 32e7dc04-98c5-4e91-a6a5-f6e676a9494c with color darkblue
Adding trip: 3309338e-7dfd-4856-baf0-816cf2fa0e21 with color pink
Adding trip: 392aad18-40b7-4f25-ad71-80d67ae69e17 with color purple
Adding trip: 3c831b33-859e-448e-88f7-61a3883daa68 with color beige
Adding trip: 3d484004-8fbe-4375-bb92-68eb3f603ce1 with color green
Adding trip: 45ad471b-f8ef-4c34-8faa-550569898703 with color orange
Adding trip: 564f3344-bcb9-43b7-91ca-7477a03825b9 with color lightgreen
Adding trip: 57ef0674-38fa-4919-a3f8-9dd687958378